In [15]:
import ete3
import numpy as np
from treefuns import *
from operator import itemgetter

In [5]:
def get_sp(node):
    spnm = node.split('_', 1)[1]
    return spnm

In [6]:
leafdists = list()
for leaf in tree.get_leaf_names():
    leafdists.append(tree.get_distance(leaf))

print((np.median(leafdists), {'wdth': tree.get_farthest_leaf()[1],
                              'len_med': np.median(leafdists),
                              'len_avg': np.mean(leafdists)}))

NameError: name 'tree' is not defined

In [7]:
treel = open('../data/0076_108.txt', 'r').read().split('\n')[0].split('\t')
t = ete3.PhyloTree(treel[3], sp_naming_function=get_sp)

In [8]:
t.set_outgroup(t.get_midpoint_outgroup())

In [9]:
t.get_descendant_evol_events()

In [19]:
parlist = list()

# Analyse all the subtrees from the tree
for st in t.traverse():
    # Get those subtrees that root in a duplication node
    if len(st.get_leaf_names()) > 1 and st.evoltype == 'D':
        childs = st.get_children()
        
        # Check both child clades have more than 4 and there are 2 childs
        if len(childs) == 2 and [len(ch.get_leaf_names()) > 4 for ch in childs]:
            chevtypes = list()
            # Go through childs storing the descendant node evoltytpe
            for child in childs:
                chevtypes += [nd.evoltype for nd in child.iter_search_nodes() 
                              if 'evoltype' in list(nd.features)]
            
            # Check all the descendant nodes from the childs are speciation
            # nodes and then store them in a list of tupples
            if set(chevtypes) == {'S'}:
                for child in st.get_children():
                    chstats = tree_stats(child)
                    parlist.append((chstats['median'], chstats))

parlist = sorted(parlist, key=itemgetter(0))
print(parlist)

[(0.15922160000000002, {'leafno': 13, 'median': 0.15922160000000002, 'mean': 0.18346275384615382, 'width': 0.4262566, 'sum': 2.3850157999999992, 'kurt': 2.769643454381928, 'skew': 1.7485548877815096}), (0.2485555, {'leafno': 9, 'median': 0.2485555, 'mean': 0.24089553333333333, 'width': 0.49819399999999997, 'sum': 2.1680598, 'kurt': -0.8912959637721558, 'skew': 0.5984124506840857})]


In [30]:
print({**{'min_' + k: v for k, v in parlist[0][1].items()},
       **{'max_' + k: v for k, v in parlist[0][1].items()}})

{'min_leafno': 13, 'min_median': 0.15922160000000002, 'min_mean': 0.18346275384615382, 'min_width': 0.4262566, 'min_sum': 2.3850157999999992, 'min_kurt': 2.769643454381928, 'min_skew': 1.7485548877815096, 'max_leafno': 13, 'max_median': 0.15922160000000002, 'max_mean': 0.18346275384615382, 'max_width': 0.4262566, 'max_sum': 2.3850157999999992, 'max_kurt': 2.769643454381928, 'max_skew': 1.7485548877815096}


In [304]:
def get_subtree(tree, feature, leafth, seed):
    vertleaves = [l for l in t.get_leaves() if str(getattr(l, feature)) != 'nan']
    leafno = int((1 - leafth) * len(t.get_leaf_names()))
    to_save = random.choices(vertleaves, k=leafno) + [seed]
    tree.prune(to_save)
    return 0

In [305]:
def get_distdict(seed, tree, subtree):
    odict = dict()
    odict[subtree + '_seed'] = seed
    odict[subtree + '_species'] = get_species_tag(seed)
    odict[subtree + '_vert_dist'] = vert_dict['node'].get_distance(seed)
    odict[subtree + '_met_dist'] = met_dict['node'].get_distance(seed)
    odict[subtree + '_seed_dist'] = tree.get_distance(seed)
    odict[subtree + '_vert_ndist'] = vert_dict['node'].get_distance(seed) / nfactor
    odict[subtree + '_met_ndist'] = met_dict['node'].get_distance(seed) / nfactor
    odict[subtree + '_seed_ndist'] = t.get_distance(seed) / nfactor
    
    return odict

In [312]:
import random
import ete3
import numpy as np
from clade_sp_dist import *

phylome_id = '0076'

odict = dict()

treel = open('../data/0076_108.txt', 'r').read().split('\n')[0].split('\t')
t = ete3.PhyloTree(treel[3], sp_naming_function=get_sp)

gnmdf = pd.read_csv('../data/0076_norm_groups.csv')
root(t, root_dict[int(phylome_id)])
t.get_descendant_evol_events()

annotate_lineages(t, gnmdf, ['Normalising group',
                             'Vertebrate',
                             'Metazoan'])

norm_dict = clade_norm(t, treel[0], 'Normalising group')

nfactor = norm_dict['norm_factor']

vert_dict = get_group_mrca(t, treel[0], 'Vertebrate',
                           'vertebrate', treel[0])
met_dict = get_group_mrca(t, treel[0], 'Metazoan',
                          'metazoan', treel[0])

odict = get_distdict(treel[0], t, 'whole')

for i in [0.1, 0.25, 0.5]:
    st = t
    
    get_subtree(st, 'Vertebrate', i, treel[0])
    
    print([l.Vertebrate for l in st.get_leaves()])
    
    norm_dict = clade_norm(st, treel[0], 'Normalising group')

    nfactor = norm_dict['norm_factor']
    
    vert_dict = get_group_mrca(t, treel[0], 'Vertebrate',
                           'vertebrate', treel[0])
    
    met_dict = get_group_mrca(st, treel[0], 'Metazoan',
                              'metazoan', treel[0])
    
    odict = {**odict, **get_distdict(treel[0], st, str(i))}

olist()

['vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates']
['vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates']
['vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates', 'vertebrates']


,whole_seed,whole_species,whole_vert_dist,whole_met_dist,whole_seed_dist,whole_vert_ndist,whole_met_ndist,whole_seed_ndist,0.1_seed,0.1_species,...,0.25_met_ndist,0.25_seed_ndist,0.5_seed,0.5_species,0.5_vert_dist,0.5_met_dist,0.5_seed_dist,0.5_vert_ndist,0.5_met_ndist,0.5_seed_ndist
0,Phy001R83G_HUMAN,HUMAN,0.432531,1.257411,2.478446,0.320156,0.930724,1.834523,Phy001R83G_HUMAN,HUMAN,...,0.979082,1.44589,Phy001R83G_HUMAN,HUMAN,0.140227,0.140227,0.248454,1.0,1.0,1.771799


In [226]:
print(stt)features


--Phy003JP2O_BOVIN


In [33]:
if (True and True):
    print('Miau')

Miau
